# Chopsticks Game AI Agent

## Project Overview
This project implements an AI player for a modified version of the children's hand game "Chopsticks" (also known as "Fingers"). The AI uses the minimax search algorithm with alpha-beta pruning to find optimal moves in the game. This implementation focuses on a 4-hand variant of the game, where two players each control two hands (players A,B vs C,D).

The core components include:
- Game state representation and legal move generation
- State transition logic
- Terminal state and utility calculations
- Heuristic evaluation function for non-terminal states
- Alpha-beta pruning implementation for efficient search

## The Game of Chopsticks

### Basic Rules
Chopsticks is traditionally played with two players, each using their two hands. In this implementation:
- Players control two hands each (A,B vs C,D)
- Each hand starts with 1 finger raised
- Players take turns tapping an opponent's hand with one of their hands
- When a hand is tapped, the number of fingers on the tapped hand increases by the number of fingers on the tapping hand
- If the sum exceeds 5, that hand is considered "dead" (value becomes 0)
- A player loses when both of their hands are dead (value 0)

### Special Move: Splitting
This variant includes a special "splitting" move:
- If a player has at least 4 fingers on one hand and their other hand is dead (value 0)
- They can perform a split, dividing the fingers between both hands
- Example: [4,0] → [2,2] or [5,0] → [3,2]

This splitting move makes the game much less predictable, and harder for an algorithm or human to solve.

## Technical Implementation

The implementation uses the `games4e` library which provides a framework for implementing adversarial search algorithms while forcing you to implement the algorithmic design.

The library is from the Python code for the book *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)

### Code Implementation

Now, let's look at the implementation of our Chopsticks game agent:

In [22]:
# Import required libraries
from games4e import *
import math

class Chopsticks(Game):
    def __init__(self, board):
        if type(board) == str: #converts state to dictionary if not already
            self.board = self.convert_state_to_dictionary(board)
        else:
            self.board = board
        moves = ['AB', 'AC', 'AD', 'BA', 'BC', 'BD', 'CD', 'CB', 'CA', 'DC', 'DB', 'DA']
        #assumes it is our turn
        self.initial = GameState(to_move=True, utility=0, board=self.board, moves=moves)

    def convert_state_to_dictionary(self, board_str):
        board = {'A': int(board_str[1]), 'B': int(board_str[3]), 'C': int(board_str[5]), 'D': int(board_str[7])}
        return board
    def actions(self, state):
        board = state.board
        to_move = state.to_move
        l_moves = []
        if to_move:  # Our turn (hands A and B)
            for move in ['AB', 'AC', 'AD', 'BA', 'BC', 'BD']:
                tapping_hand = move[0]
                tapped_hand = move[1]
                if board[tapping_hand] == 0:  # Can't use a dead hand
                    continue
                elif board[tapped_hand] == 0 and board[tapping_hand] >= 4 and move in ['AB', 'BA']:
                    l_moves.append(move)  # Special splitting move
                elif board[tapped_hand] == 0:  # Can't tap a dead hand (unless splitting)
                    continue
                else:
                    if move not in ['AB', 'BA']:  # Regular tapping move against opponent
                        l_moves.append(move)
        else:  # Opponent's turn (hands C and D)
            for move in ['CD', 'CB', 'CA', 'DC', 'DB', 'DA']:
                tapping_hand = move[0]
                tapped_hand = move[1]
                if board[tapping_hand] == 0:
                    continue
                elif board[tapped_hand] == 0 and board[tapping_hand] >= 4 and move in ['CD', 'DC']:
                    l_moves.append(move)  # Special splitting move
                elif board[tapped_hand] == 0:
                    continue
                else:
                    if move not in ['CD', 'DC']:  # Regular tapping move against opponent
                        l_moves.append(move)
        return l_moves
    def result(self, state, move):
        ''' Returns the state that results after making a move'''
        if move not in self.actions(state):
            return state
        tapping_hand = move[0]
        tapped_hand = move[1]
        new_board = state.board.copy()
        if state.board[tapped_hand] == 0: #checks if move is split
            new_board[tapped_hand] = math.floor(state.board[tapping_hand]/2)
            new_board[tapped_hand] = math.ceil(state.board[tapping_hand]/2)
        else: 
            if state.board[tapped_hand] + state.board[tapping_hand] > 5:
                new_board[tapped_hand] = 0
            else:
                new_board[tapped_hand] = state.board[tapping_hand] + state.board[tapped_hand]
        moves = state.moves
        return GameState(to_move=not state.to_move, 
                         utility=self.compute_utility(new_board, move, state.to_move),
                         board=new_board, moves=moves)    
# Continuing Chopsticks class definition - Game termination and utility methods
    def utility(self, state, to_move):
        ''' Returns the utility of the current state'''
        return state.utility if to_move else -state.utility
    
    def terminal_test(self, state):
        """A state is terminal if it is won
        Returns a boolean value
        """
        if (state.board['C'] == 0 and state.board['D'] == 0) or (state.board['A'] == 0 and state.board['B'] == 0):
            return True
        return len(state.moves) == 0

    def next_board(self, board, move):
        ''' Returns the board that results after making a move'''
        tapping_hand = move[0]
        tapped_hand = move[1]
        new_board = board.copy()
        if board[tapped_hand] == 0: #checks if move is split
            new_board[tapped_hand] = math.floor(board[tapping_hand]/2)
            new_board[tapped_hand] = math.ceil(board[tapping_hand]/2)
        else: 
            if board[tapped_hand] + board[tapped_hand] > 5:
                new_board[tapped_hand] = 0
            else:
                new_board[tapped_hand] = board[tapping_hand] + board[tapped_hand]        
        return new_board

    def compute_utility(self,board,move,to_move):
        """If 'X' wins with this move, return INF; if 'O' wins return -INF; 
        else return 0."""
        next_board = self.next_board(board,move)
        to_move = not to_move
        if to_move and (next_board['C'] == 0 and next_board['D'] == 0):
            return math.inf
            #else if terminal and winning for opponent, return negative Inf
        elif not to_move and (next_board['A'] == 0 and next_board['B'] == 0):
            return -math.inf
        else:
            return 0
    def estimate_utility(self, state):
        ''' Returns the estimated utility of the state which has no children and is not in terminal state 
        using a heuristic function
        '''
        h = 0
        if self.terminal_test(state):
            if (state.board['C'] == 0 and state.board['D'] == 0):
                return math.inf
            else: 
                return -math.inf 
        # Evaluate advantage based on number of active hands
        if state.board['C'] >= 2 and state.board['D'] >= 2 and (state.board['A'] == 0 or state.board['B'] == 0): 
            h -= 10  # Disadvantage: opponent has 2 hands vs our 1 hand
        elif (state.board['C'] == 0 or state.board['D'] == 0) and state.board['A'] >= 2 and state.board['B'] >= 2: 
            h += 10  # Advantage: we have 2 hands vs opponent's 1 hand
        
        # Value of dead hands
        if state.board['C'] == 0 or state.board['D'] == 0: 
            h += 3  # Positive value for opponents having dead hands
        if state.board['A'] == 0 or state.board['B'] == 0: 
            h -= 3  # Negative value for our hands being dead
        
        # Tactical positions
        if (state.board['A'] == 5 or state.board['B'] == 5) and state.to_move and state.board['A'] >= 2 and state.board['B'] >= 2: 
            h += 10  # Advantage: having a hand with 5 fingers on our turn
        elif (state.board['C'] == 5 or state.board['D'] == 5) and not state.to_move and state.board['C'] >= 2 and state.board['D'] >= 2:
            h -= 10  # Disadvantage: opponent having a hand with 5 fingers on their turn
            
        return h
def alpha_beta_cutoff_search(state, game, d=12, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""
    
    if eval_fn is None:
        eval_fn = game.estimate_utility

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = -math.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = math.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Quick check for immediate winning moves
    for a in game.actions(state):
        next_state = game.result(state,a)
        if next_state.board['C'] == 0 and next_state.board['D'] == 0:
            return a
    
    # Main search with alpha-beta pruning
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    best_score = -math.inf
    beta = math.inf
    best_action = None
    
    actions = game.actions(state)
    if actions:
        best_action = actions[0]  # Default to first available action
        for a in actions:
            v = max_value(game.result(state, a), best_score, beta, 1)
            if v > best_score:
                best_score = v
                best_action = a
    return best_action

def alpha_beta_cutoff_search(state, game, d=12, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""
    
    if eval_fn is None:
        eval_fn = game.estimate_utility

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = -math.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = math.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Quick check for immediate winning moves
    for a in game.actions(state):
        next_state = game.result(state,a)
        if next_state.board['C'] == 0 and next_state.board['D'] == 0:
            return a
    
    # Main search with alpha-beta pruning
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    best_score = -math.inf
    beta = math.inf
    best_action = None
    
    actions = game.actions(state)
    if actions:
        best_action = actions[0]  # Default to first available action
        for a in actions:
            v = max_value(game.result(state, a), best_score, beta, 1)
            if v > best_score:
                best_score = v
                best_action = a
    return best_action

### Generating Legal Moves

The `actions` method determines all legal moves available from a given game state. Moves are represented as two-character strings where:
- First character: The tapping hand (A, B, C, or D)
- Second character: The tapped hand (A, B, C, or D)

The method handles different rules for each player and accounts for special splitting moves.

### State Transitions

The `result` method computes the new state that results from making a move. It:
1. Validates that the move is legal
2. Determines if the move is a regular tap or a split
3. Updates the game board accordingly
4. Computes the utility of the new state
5. Returns a new GameState object

### Game Termination and Utility

These methods determine when the game has ended and calculate the utility (value) of a state:
- `utility`: Returns the utility value of a state from a player's perspective
- `terminal_test`: Checks if a state is terminal (game over)
- `next_board`: Helper method for calculating the next board state
- `compute_utility`: Calculates the utility of a state after a move

### Heuristic Evaluation

For non-terminal states, we need a heuristic function to estimate their value. The `estimate_utility` method evaluates a state based on several factors:
1. Terminal states (win/loss)
2. Advantage in number of active hands
3. Dead hands for either player
4. Tactical positions (having a hand with 5 fingers)

This heuristic guides the search algorithm toward promising moves when it can't search to the end of the game.

## The Search Algorithm: Alpha-Beta Pruning

The alpha-beta pruning algorithm efficiently searches the game tree for the best move. It:
1. Explores possible moves and their consequences
2. Prunes (skips) branches that are provably worse than already-explored options
3. Uses a depth limit and heuristic evaluation for efficiency
4. Prioritizes moves that lead to immediate wins

## Main Interface Function

The `generate_move` function serves as the main interface for the AI agent. Given a board state string, it:
1. Creates a game instance
2. Calls the alpha-beta search algorithm with a depth limit of 11
3. Returns the optimal move

In [23]:
def generate_move(str_board):
    """
    Main function to generate the best move given a board state string.
    
    Args:
        str_board (str): A string representation of the board state in format "[A][B][C][D]"
                         where each letter represents the number of fingers on that hand
    
    Returns:
        str: The best move as a two-letter string (e.g., "AC" for hand A tapping hand C)
    """
    f_game = Chopsticks(str_board)
    return str(alpha_beta_cutoff_search(f_game.initial, f_game, d=11, cutoff_test=None, eval_fn=f_game.estimate_utility))

## Demo: Playing the Game

Let's demonstrate how the agent plays the game by showing its decisions in different board states:

In [24]:
# Demo: Generate moves for different board states
test_states = [
    "[1,1,1,1]",  # Initial state
    "[1,2,0,2]",  # Mid-game state
    "[0,4,1,3]",  # Our first hand is dead
    "[2,3,0,2]",  # Opponent's first hand is dead
    "[4,0,1,2]"   # Potential splitting move
]

print("Board State | Best Move | Description")
print("------------|-----------|------------")

for state in test_states:
    board = Chopsticks(state).board
    best_move = generate_move(state)
    
    # Create a description of the move
    tapping_hand = best_move[0]
    tapped_hand = best_move[1]
    
    if board[tapped_hand] == 0 and tapping_hand in ['A', 'B'] and tapped_hand in ['A', 'B']:
        description = f"Split hand {tapping_hand}'s {board[tapped_hand]} fingers"
    else:
        description = f"Tap {tapped_hand} with {tapping_hand}"
    
    print(f"{state} | {best_move} | {description}")

Board State | Best Move | Description
------------|-----------|------------
[1,1,1,1] | AC | Tap C with A
[1,2,0,2] | AD | Tap D with A
[0,4,1,3] | BD | Tap D with B
[2,3,0,2] | AD | Tap D with A
[4,0,1,2] | AD | Tap D with A


## Visualizing the Game State

Let's create a simple visualization function to better understand the game state:

In [25]:
def visualize_state(board_str):
    """Visualize the game state in a more human-readable format"""
    if isinstance(board_str, str):
        board = Chopsticks(board_str).board
    else:
        board = board_str
    
    # Unicode fingers: 👆✌👌🖖🖐️
    finger_symbols = {0: "✊", 1: "👆", 2: "✌", 3: "👌", 4: "🖖", 5: "🖐️"}
    
    print("Player 1 (our agent):")
    print(f"Hand A: {finger_symbols.get(board['A'], '?')} ({board['A']})")
    print(f"Hand B: {finger_symbols.get(board['B'], '?')} ({board['B']})")
    print("\nPlayer 2 (opponent):")
    print(f"Hand C: {finger_symbols.get(board['C'], '?')} ({board['C']})")
    print(f"Hand D: {finger_symbols.get(board['D'], '?')} ({board['D']})")

# Try the visualization with different states
for state in test_states:
    print(f"\nVisualizing state: {state}")
    visualize_state(state)
    print(f"Best move: {generate_move(state)}")


Visualizing state: [1,1,1,1]
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: 👆 (1)

Player 2 (opponent):
Hand C: 👆 (1)
Hand D: 👆 (1)
Best move: AC

Visualizing state: [1,2,0,2]
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: ✌ (2)

Player 2 (opponent):
Hand C: ✊ (0)
Hand D: ✌ (2)
Best move: AD

Visualizing state: [0,4,1,3]
Player 1 (our agent):
Hand A: ✊ (0)
Hand B: 🖖 (4)

Player 2 (opponent):
Hand C: 👆 (1)
Hand D: 👌 (3)
Best move: BD

Visualizing state: [2,3,0,2]
Player 1 (our agent):
Hand A: ✌ (2)
Hand B: 👌 (3)

Player 2 (opponent):
Hand C: ✊ (0)
Hand D: ✌ (2)
Best move: AD

Visualizing state: [4,0,1,2]
Player 1 (our agent):
Hand A: 🖖 (4)
Hand B: ✊ (0)

Player 2 (opponent):
Hand C: 👆 (1)
Hand D: ✌ (2)
Best move: AD


## Simulating a Game

Let's simulate a complete game to see how the AI agent plays:

In [26]:
def simulate_game(initial_state="[1,1,1,1]", max_moves=30):
    """Simulate a game where our agent plays against a simplified opponent"""
    board = Chopsticks(initial_state).board
    game = Chopsticks(board)
    current_state = game.initial
    
    print("Starting a new game simulation!")
    print("Initial state:")
    visualize_state(board)
    
    move_count = 0
    while not game.terminal_test(current_state) and move_count < max_moves:
        move_count += 1
        
        # Our turn
        if current_state.to_move:
            # Convert board to string format for generate_move
            board_str = f"[{board['A']},{board['B']},{board['C']},{board['D']}]"
            move = generate_move(board_str)
            print(f"\nMove {move_count} (Our agent): {move}")
        # Opponent's turn - using a simple strategy (first valid move)
        else:
            move = game.actions(current_state)[0]
            print(f"\nMove {move_count} (Opponent): {move}")
        
        # Apply the move
        current_state = game.result(current_state, move)
        board = current_state.board
        
        # Display the new state
        visualize_state(board)
        
        # Check for winner
        if game.terminal_test(current_state):
            if board['C'] == 0 and board['D'] == 0:
                print("\nGame Over: Our agent wins! 🎉")
            elif board['A'] == 0 and board['B'] == 0:
                print("\nGame Over: Opponent wins! 😢")
            else:
                print("\nGame Over: No legal moves left!")
            break
    
    if move_count >= max_moves:
        print("\nGame reached maximum number of moves without conclusion.")
    
    return board

# Run a game simulation
simulate_game()

Starting a new game simulation!
Initial state:
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: 👆 (1)

Player 2 (opponent):
Hand C: 👆 (1)
Hand D: 👆 (1)

Move 1 (Our agent): AC
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: 👆 (1)

Player 2 (opponent):
Hand C: ✌ (2)
Hand D: 👆 (1)

Move 2 (Opponent): CB
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: 👌 (3)

Player 2 (opponent):
Hand C: ✌ (2)
Hand D: 👆 (1)

Move 3 (Our agent): AD
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: 👌 (3)

Player 2 (opponent):
Hand C: ✌ (2)
Hand D: ✌ (2)

Move 4 (Opponent): CB
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: 🖐️ (5)

Player 2 (opponent):
Hand C: ✌ (2)
Hand D: ✌ (2)

Move 5 (Our agent): AC
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: 🖐️ (5)

Player 2 (opponent):
Hand C: 👌 (3)
Hand D: ✌ (2)

Move 6 (Opponent): CB
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: ✊ (0)

Player 2 (opponent):
Hand C: 👌 (3)
Hand D: ✌ (2)

Move 7 (Our agent): AC
Player 1 (our agent):
Hand A: 👆 (1)
Hand B: ✊ (0)

Player 2 (opponent):
Hand C: 🖖 (4)


{'A': 5, 'B': 0, 'C': 4, 'D': 2}

## Performance and Analysis

The AI agent uses alpha-beta pruning with a depth limit of 11 moves, allowing it to make decisions quickly while still playing at a high level. The heuristic evaluation function is crucial for guiding the search when it can't reach terminal states.

### Strengths:
- **Immediate win detection**: The agent will always choose a winning move if available
- **Strategic evaluation**: Considers hand advantages, dead hands, and tactical positions
- **Efficient search**: Alpha-beta pruning dramatically reduces the search space

### Areas for improvement:
- **Advanced heuristics**: Could incorporate more pattern recognition
- **Learning capability**: Could adapt to opponent play styles over time
- **Deeper search**: With optimization, could search deeper into the game tree

The current implementation achieves approximately 70th percentile performance in competitions against other AI agents.

## Conclusion

This project demonstrates the application of adversarial search algorithms to create an AI agent for the game of Chopsticks. The implementation showcases:

1. Game state representation and manipulation
2. Minimax search with alpha-beta pruning
3. Heuristic evaluation for non-terminal states
4. Search optimization techniques

The resulting agent plays at a competitive level while maintaining efficiency. This approach could be extended to other turn-based games with complete information.

## Future Work

Potential enhancements include:
- Implementing machine learning approaches (reinforcement learning)
- Developing opening move databases
- Optimizing search for even deeper exploration

## Interactive GUI: Play Against the AI

Now let's create an interactive GUI using IPython widgets to allow users to play directly against our AI agent within this notebook. This interface will:

1. Display the current state of the game using our finger emojis
2. Allow the user to select their move using dropdown menus
3. Show the AI's response move
4. Keep track of game history and display the winner

First, let's install and import the required libraries:

In [27]:
# Install ipywidgets if needed
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except ImportError:
    !pip install ipywidgets
    import ipywidgets as widgets
    from IPython.display import display, clear_output

In [ ]:
class ChopsticksGUI:
    def __init__(self):
        # Initialize game state
        self.initial_state = "[1,1,1,1]"
        self.game = Chopsticks(self.initial_state)
        self.board = self.game.board
        self.current_state = self.game.initial
        self.game_log = [f"Game started with state: {self.initial_state}"]
        self.game_over = False
        
        # Finger symbols
        self.finger_symbols = {0: "✊", 1: "👆", 2: "✌", 3: "👌", 4: "🖖", 5: "🖐️"}
        
        # Create widgets
        self.output = widgets.Output(layout={'border': '1px solid black', 'height': '300px', 'overflow_y': 'auto'})
        self.status_label = widgets.HTML(value="<h3>Your Turn</h3>")
        
        # Player hand selection (which hand to tap with)
        self.hand_label = widgets.Label('Select your hand:')
        self.hand_dropdown = widgets.Dropdown(
            options=[('Hand C', 'C'), ('Hand D', 'D')],
            value='C',
            description='From:',
            disabled=False
        )
        
        # Target hand selection (which hand to tap)
        self.target_label = widgets.Label('Select target hand:')
        self.target_dropdown = widgets.Dropdown(
            options=[('Hand A', 'A'), ('Hand B', 'B')],
            value='A',
            description='To:',
            disabled=False
        )
        
        # Set up the hand dropdown change event
        self.hand_dropdown.observe(self.on_hand_changed, names='value')
        
        # Move button
        self.move_button = widgets.Button(
            description='Make Move',
            disabled=False,
            button_style='success',
            tooltip='Click to make your move'
        )
        self.move_button.on_click(self.on_move_button_clicked)
        
        # Reset button
        self.reset_button = widgets.Button(
            description='Reset Game',
            disabled=False,
            button_style='warning',
            tooltip='Click to reset the game'
        )
        self.reset_button.on_click(self.reset_game)
        
        # Layout
        self.hand_box = widgets.HBox([self.hand_label, self.hand_dropdown])
        self.target_box = widgets.HBox([self.target_label, self.target_dropdown])
        self.controls = widgets.VBox([self.hand_box, self.target_box, 
                                    widgets.HBox([self.move_button, self.reset_button])])
        
        # Main layout
        self.main_box = widgets.VBox([self.status_label, self.controls, self.output])
        
        # Display initial state
        self.update_display()
        
        # If it's AI's turn on startup, make its move immediately
        if self.current_state.to_move and not self.game_over:
            self.make_ai_move()
            self.update_display()

    def get_valid_moves(self):
        """Get valid moves for the current player"""
        valid_moves = []
        moves = self.game.actions(self.current_state)
        
        # Filter moves for the human player (hands C and D)
        if not self.current_state.to_move:  # Human's turn
            valid_hand_targets = {}
            for move in moves:
                hand = move[0]
                target = move[1]
                
                # Create nested dict structure
                if hand not in valid_hand_targets:
                    valid_hand_targets[hand] = []
                valid_hand_targets[hand].append(target)
            
            return valid_hand_targets
        return {}
    
    def update_dropdowns(self):
        """Update dropdown options based on valid moves"""
        valid_moves = self.get_valid_moves()
        
        # If no valid moves, disable controls
        if not valid_moves:
            self.hand_dropdown.options = [('Hand C', 'C'), ('Hand D', 'D')]
            self.target_dropdown.options = [('Hand A', 'A'), ('Hand B', 'B')]
            return
        
        # Update hand dropdown
        hands = list(valid_moves.keys())
        if hands:
            self.hand_dropdown.options = [(f"Hand {h}", h) for h in hands]
            self.hand_dropdown.value = hands[0]
            
            # Update target dropdown based on selected hand
            self.update_target_dropdown()
    
    def on_hand_changed(self, change):
        """Handler for when the hand dropdown selection changes"""
        self.update_target_dropdown()
        
    def update_target_dropdown(self):
        """Update target dropdown based on selected hand"""
        valid_moves = self.get_valid_moves()
        selected_hand = self.hand_dropdown.value
        
        if selected_hand in valid_moves:
            targets = valid_moves[selected_hand]
            self.target_dropdown.options = [(f"Hand {t}", t) for t in targets]
            if targets:
                self.target_dropdown.value = targets[0]
    
    def on_move_button_clicked(self, b):
        """Handle move button clicks"""
        if self.game_over:
            with self.output:
                clear_output()
                print("Game is over! Please reset to play again.")
            return
        
        # Get the selected move
        from_hand = self.hand_dropdown.value
        to_hand = self.target_dropdown.value
        move = from_hand + to_hand
        
        # Make human move
        self.make_move(move)
        
        # If game isn't over, make AI move
        if not self.game_over and self.current_state.to_move:
            self.make_ai_move()
            
        # Update display
        self.update_display()
    
    def make_move(self, move):
        """Make a move and update the game state"""
        # Check if move is valid
        if move not in self.game.actions(self.current_state):
            with self.output:
                print(f"Invalid move: {move}")
            return False
        
        # Record the move
        mover = "You" if not self.current_state.to_move else "AI"
        description = self.get_move_description(move)
        self.game_log.append(f"{mover} played: {move} ({description})")
        
        # Apply the move
        self.current_state = self.game.result(self.current_state, move)
        self.board = self.current_state.board
        
        # Check for game over
        if self.game.terminal_test(self.current_state):
            if self.board['A'] == 0 and self.board['B'] == 0:
                self.game_log.append("Game Over: You win! 🎉")
            elif self.board['C'] == 0 and self.board['D'] == 0:
                self.game_log.append("Game Over: AI wins! 🤖")
            else:
                self.game_log.append("Game Over: No legal moves left!")
            self.game_over = True
            
        return True
    
    def make_ai_move(self):
        """Let the AI make its move"""
        # Convert board to string format for generate_move
        board_str = f"[{self.board['A']},{self.board['B']},{self.board['C']},{self.board['D']}]"
        ai_move = generate_move(board_str)
        
        # Make the move
        self.make_move(ai_move)
    
    def get_move_description(self, move):
        """Get a human-readable description of a move"""
        tapping_hand = move[0]
        tapped_hand = move[1]
        
        if self.board[tapped_hand] == 0 and ((tapping_hand in ['A', 'B'] and tapped_hand in ['A', 'B']) or 
                                            (tapping_hand in ['C', 'D'] and tapped_hand in ['C', 'D'])):
            return f"Split hand {tapping_hand}'s {self.board[tapped_hand]} fingers"
        else:
            return f"Tap {tapped_hand} with {tapping_hand}"
    
    def update_display(self):
        """Update the game display"""
        with self.output:
            clear_output()
            
            # Display board state
            print("Current Game State:")
            print("\nAI Player:")
            print(f"Hand A: {self.finger_symbols.get(self.board['A'], '?')} ({self.board['A']})")
            print(f"Hand B: {self.finger_symbols.get(self.board['B'], '?')} ({self.board['B']})")
            print("\nYou (Human Player):")
            print(f"Hand C: {self.finger_symbols.get(self.board['C'], '?')} ({self.board['C']})")
            print(f"Hand D: {self.finger_symbols.get(self.board['D'], '?')} ({self.board['D']})")
            
            print("\nGame Log:")
            for entry in self.game_log[-5:]:  # Show last 5 entries
                print(entry)
                
            if self.game_over:
                print("\n🔄 Press 'Reset Game' to play again")
        
        # Update status label
        if self.game_over:
            if self.board['A'] == 0 and self.board['B'] == 0:
                self.status_label.value = "<h3 style='color:green'>You Win! 🎉</h3>"
            elif self.board['C'] == 0 and self.board['D'] == 0:
                self.status_label.value = "<h3 style='color:red'>AI Wins! 🤖</h3>"
            else:
                self.status_label.value = "<h3 style='color:blue'>Game Over - No Moves Left!</h3>"
        elif self.current_state.to_move:
            self.status_label.value = "<h3 style='color:blue'>AI's Turn (thinking...)</h3>"
        else:
            self.status_label.value = "<h3 style='color:green'>Your Turn</h3>"
            
        # Update dropdown options
        self.update_dropdowns()
            
        # Enable/disable controls based on game state
        self.move_button.disabled = self.game_over or self.current_state.to_move
    
    def reset_game(self, b=None):
        """Reset the game to initial state"""
        self.game = Chopsticks(self.initial_state)
        self.board = self.game.board
        self.current_state = self.game.initial
        self.game_log = [f"Game started with state: {self.initial_state}"]
        self.game_over = False
        
        # Update display
        self.update_display()
    
    def display(self):
        """Display the game GUI"""
        display(self.main_box)
        self.update_display()

# Create and display the game
try:
    # Clean up any previous instances
    del game_gui
except NameError:
    pass

game_gui = ChopsticksGUI()
game_gui.display()

### How to Use the GUI

1. **Starting the Game**: You play as the hands C and D against the AI (hands A and B).
2. **Making a Move**: 
   - Choose which of your hands to use from the "From" dropdown
   - Choose which of the AI's hands to target from the "To" dropdown
   - Click "Make Move" to execute your move
3. **Game Progress**: 
   - The game log shows recent moves and events
   - The current state is visualized with finger emojis
   - The status bar indicates whose turn it is
4. **Game End**:
   - The game ends when either player has both hands "dead" (0 fingers)
   - Click "Reset Game" to start a new game

This interface provides a convenient way to test the AI agent's capabilities and understand its strategy through direct gameplay. Try different opening moves and see if you can find a strategy to beat the AI!